In [25]:
import qiskit
import qiskit_aer
import qiskit_ibm_runtime
import qiskit_machine_learning
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import os
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

In [27]:
original_path = "Test/Test"

if(not os.path.exists(original_path + "/face_pixels")):
    os.mkdir(path= original_path + "/face_pixels")

if(not os.path.exists(original_path + "/face_extracted")):
    os.mkdir(path= original_path + "/face_extracted")

for folder in os.listdir(original_path):
    if(folder != ".DS_Store" and folder != "face_pixels" and folder != "face_extracted"):
        folder_path = original_path + "/" + folder
        for image in os.listdir(folder_path):
            if(image != ".DS_Store"):
                face_objs = DeepFace.extract_faces(img_path=folder_path + "/" + image, detector_backend="mtcnn", align=True, enforce_detection=False)
                for face_obj in face_objs:
                    if(not os.path.exists(original_path + "/face_extracted/" + folder)):
                        os.mkdir(path= original_path + "/face_extracted/" + folder)

                    plt.imsave(original_path + "/face_extracted/" + folder + "/" + image, face_obj['face'])

                    if(not os.path.exists(original_path + "/face_pixels/" + folder)):
                        os.mkdir(path= original_path + "/face_pixels/" + folder)
    
                    np.save(original_path + "/face_pixels/" + folder + "/" + image.split(".")[0] + ".npy", face_obj['face'])


In [21]:
original_path = "Dataset/Dataset/val"

if(not os.path.exists(original_path + "/face_pixels")):
    os.mkdir(path= original_path + "/face_pixels")

if(not os.path.exists(original_path + "/face_extracted")):
    os.mkdir(path= original_path + "/face_extracted")

for folder in os.listdir(original_path):
    if(folder != ".DS_Store" and folder != "face_pixels" and folder != "face_extracted"):
        folder_path = original_path + "/" + folder
        for image in os.listdir(folder_path):
            face_objs = DeepFace.extract_faces(img_path=folder_path + "/" + image, detector_backend="mtcnn", align=True, enforce_detection=False)
            for face_obj in face_objs:
                if(not os.path.exists(original_path + "/face_extracted/" + folder)):
                    os.mkdir(path= original_path + "/face_extracted/" + folder)

                plt.imsave(original_path + "/face_extracted/" + folder + "/" + image, face_obj['face'])

                if(not os.path.exists(original_path + "/face_pixels/" + folder)):
                    os.mkdir(path= original_path + "/face_pixels/" + folder)

                np.save(original_path + "/face_pixels/" + folder + "/" + image.split(".")[0] + ".npy", face_obj['face'])

### Embedding Dataset

In [29]:
face_path = "Test/Test/face_extracted"

features = []

for person in os.listdir(face_path):
    person_path = os.path.join(face_path, person)
    if os.path.isdir(person_path):
        print("helo")
        for img_file in os.listdir(person_path):
            img_path = os.path.join(person_path, img_file)
            img = cv2.imread(img_path)
            result = DeepFace.represent(img, model_name="Facenet", enforce_detection=False)
            embedding = result[0]["embedding"]
            features.append({"person": person, "embedding": embedding})

features_df = pd.DataFrame(features)
features_df.to_csv("Test/Test/face_extracted/features_val.csv",index=False)

helo
helo


In [3]:
features_df

,person,embedding
0,Person_0,"[-0.065240278840065, -0.42443472146987915, -0...."
1,Person_0,"[-0.3607255220413208, -0.7096951007843018, -0...."
2,Person_0,"[-0.819769024848938, -0.8046450614929199, -0.0..."
3,Person_0,"[-0.976752519607544, -1.0356870889663696, -0.2..."
4,Person_0,"[-0.6396281719207764, -0.7940641641616821, 0.2..."
...,...,...
1792,Person_7,"[-0.20930936932563782, -0.40932536125183105, -..."
1793,Person_7,"[-0.15438316762447357, -0.38950544595718384, -..."
1794,Person_7,"[-0.16873298585414886, -0.42339274287223816, -..."
1795,Person_7,"[-0.040292881429195404, -0.3078988790512085, -..."


In [4]:
features_df.to_csv("features.csv", index=False)

In [14]:
standardized_data = StandardScaler().fit_transform(features_df["embedding"].tolist())
print(standardized_data.shape)


(1797, 128)


In [15]:
labels = features_df["person"].tolist()

In [16]:
model = TSNE(n_components = 2, random_state = 0)


In [17]:
tsne_data = model.fit_transform(standardized_data)

In [18]:
tsne_data 

array([[ 17.907701 ,  -8.080799 ],
       [ 12.048971 , -44.16077  ],
       [  2.7890203, -44.11603  ],
       ...,
       [ 30.943037 ,  -2.5851839],
       [ 12.063909 ,  -1.824051 ],
       [ -7.504147 , -15.065622 ]], dtype=float32)

In [19]:
tsne_data = np.vstack((tsne_data.T, labels)).T
tsne_df = pd.DataFrame(data = tsne_data,
     columns =("Dim_1", "Dim_2", "person"))

tsne_df.to_csv("tsne_results.csv", index=False)


In [130]:
sample_df = tsne_df.sample(25)